In [112]:
import pandas as pd

stats_2023 = pd.read_csv("../../data/preprocess/pre-processed.csv")
stats_2023.drop(columns=["Profit", "Selling", "Revenue"], inplace=True)
stats_2023.dtypes

Field ID           int64
Field Name        object
Crop ID            int64
Crop Name         object
Crop Type         object
Planting Area    float64
Season ID          int64
Season            object
Field Type        object
Field Area       float64
Per Yield        float64
Per Cost         float64
Per Price        float64
Yield            float64
Cost             float64
dtype: object

# 初始化

In [113]:
import pulp
import numpy as np

# 定义辅助变量

In [114]:
fields_id = list(map(int, stats_2023["Field ID"].unique()))  # 所有的地块
crops_id = list(map(int, stats_2023["Crop ID"].unique()))  # 所有的作物
seasons_id = list(map(int, stats_2023["Season ID"].unique()))  # 所有的时节
fields_id.sort()
crops_id.sort()
seasons_id.sort()

years = range(2023, 2031)  # 要优化的年份


def pop_ndarray(arr, element_to_pop):
    index = np.where(arr == element_to_pop)[0]
    if index.size > 0:
        arr = np.delete(arr, index)
    return arr


# A类粮食作物的集合
grains_A = stats_2023[stats_2023["Crop Type"] == "粮食"]["Crop ID"].unique()
grains_A_beans = stats_2023[stats_2023["Crop Type"] == "粮食（豆类）"][
    "Crop ID"
].unique()
grains_A = np.concatenate([grains_A, grains_A_beans])
grains_A = pop_ndarray(grains_A, 16)
grains_A.sort()

# B类粮食作物的集合
grains_B = np.array([16])

# A类蔬菜作物的集合
vege_A = stats_2023[stats_2023["Crop Type"] == "蔬菜"]["Crop ID"].unique()
vege_A_beans = stats_2023[stats_2023["Crop Type"] == "蔬菜（豆类）"]["Crop ID"].unique()
vege_A = np.concatenate([vege_A, vege_A_beans])
vege_A.sort()

for i in range(3):
    vege_A = pop_ndarray(vege_A, 35 + i)

# B类蔬菜作物的集合
vege_B = np.array([35, 36, 37])

# 食用菌作物的集合
mush = stats_2023[stats_2023["Crop Type"] == "食用菌"]["Crop ID"].unique()
mush.sort()

# 豆类作物的集合
beans = stats_2023[
    (stats_2023["Crop Type"] == "粮食（豆类）")
    | (stats_2023["Crop Type"] == "蔬菜（豆类）")
]["Crop ID"].unique()
beans.sort()

# 第 i 个地块的类型（如平旱地、梯田、山坡地、智能大棚、普通大棚、水浇地）
t_i = {
    i: stats_2023[stats_2023["Field ID"] == i]["Field Type"].values[0]
    for i in fields_id
}

# 第 i 个地块在第 s 季可种植的作物集合
T_hat_i_s: dict[tuple : np.ndarray] = {}
water_field_first_season_choice = {}  # 记录水浇地在第一季的选择


# 随机获取水浇地在第一季的选择
def get_first_season_choice():
    import random

    return random.choice(list(np.concatenate([grains_B, vege_A])))


for i in fields_id:
    if t_i[i] in ["平旱地", "梯田", "山坡地"]:
        T_hat_i_s[(i, 1)] = grains_A
        T_hat_i_s[(i, 2)] = np.array([])
    elif t_i[i] in ["水浇地"]:
        first_season_choice = get_first_season_choice()
        water_field_first_season_choice[i] = first_season_choice
        if first_season_choice in grains_B:
            T_hat_i_s[(i, 1)] = grains_B
            T_hat_i_s[(i, 2)] = np.array([])
        elif first_season_choice in vege_A:
            T_hat_i_s[(i, 1)] = vege_A
            T_hat_i_s[(i, 2)] = vege_B
    elif t_i[i] in ["普通大棚"]:
        T_hat_i_s[(i, 1)] = vege_A
        T_hat_i_s[(i, 2)] = mush
    elif t_i[i] in ["智慧大棚"]:
        T_hat_i_s[(i, 1)] = vege_A
        T_hat_i_s[(i, 2)] = vege_A

S: dict[tuple : np.int64] = {}  # 期望产量
Y: dict[tuple : np.int64] = {}  # 单位面积产量
P: dict[tuple : np.int64] = {}  # 售价
C: dict[tuple : np.int64] = {}  # 单位面积成本

for j in crops_id:
    # 所有参数与 2023 年的保持一致
    for k in years:
        S[(j, k)] = stats_2023.groupby(["Crop ID"])["Yield"].agg("sum").values[j - 1]
        Y[(j, k)] = stats_2023[stats_2023["Crop ID"] == j]["Per Yield"].values[0]
        P[(j, k)] = stats_2023[stats_2023["Crop ID"] == j]["Per Price"].values[0]
        C[(j, k)] = stats_2023[stats_2023["Crop ID"] == j]["Per Cost"].values[0]

# 定义决策变量

In [115]:
# A^1_{ijks}：第 k 年第 s 季度的第 i 块地，第 j 作物的种植面积，若为不为大棚地块则为 0
# A^2_{ijks}：第 k 年第 s 季度的第 i 块地，第 j 作物的种植面积，若为大棚地块则为 0
# A_{ijks}：第 k 年第 s 季度的第 i 块地，第 j 作物的种植面积，若为大棚地块则为 0
normal_field = [i for i in fields_id if not t_i[i].endswith("大棚")]
tent_field = [i for i in fields_id if t_i[i].endswith("大棚")]

A1_ijks = pulp.LpVariable.dicts(
    "A1",
    [
        (i, j, k, s)
        for i in fields_id
        for j in crops_id
        for k in years
        for s in seasons_id
    ],
    lowBound=0,
    cat="Continuous",
)

A2_ijks = pulp.LpVariable.dicts(
    "A2",
    [
        (i, j, k, s)
        for i in fields_id
        for j in crops_id
        for k in years
        for s in seasons_id
    ],
    lowBound=0,
    cat="Continuous",
)

# Rename A1 and A2
for i in fields_id:
    for j in crops_id:
        for k in years:
            for s in seasons_id:
                A1_ijks[(i, j, k, s)].name = f"A1_{i}_{j}_{k}_{s}"
                A2_ijks[(i, j, k, s)].name = f"A2_{i}_{j}_{k}_{s}"
                A1_ijks[(i, j, k, s)].setInitialValue(0)
                A2_ijks[(i, j, k, s)].setInitialValue(0)

for i in fields_id:
    for j in crops_id:
        for k in years:
            for s in seasons_id:
                if i in normal_field:
                    A2_ijks[(i, j, k, s)].upBound = 0
                elif i in tent_field:
                    A1_ijks[(i, j, k, s)].upBound = 0

# k = 2023 的决策变量已知
k = 2023
for i in fields_id:
    for j in crops_id:
        for s in seasons_id:

            matching_rows = stats_2023[
                (stats_2023["Field ID"] == i)
                & (stats_2023["Crop ID"] == j)
                & (stats_2023["Season ID"] == s)
            ]

            area = (
                matching_rows["Planting Area"].values[0]
                if not matching_rows.empty
                else 0
            )

            if i in normal_field:
                A1_ijks[(i, j, k, s)].name = f"A1_{i}_{j}_{k}_{s}"
                A1_ijks[(i, j, k, s)].lowBound = 0
                A1_ijks[(i, j, k, s)].upBound = 0
                A1_ijks[(i, j, k, s)].cat = "Continuous"

                A2_ijks[(i, j, k, s)].name = f"A2_{i}_{j}_{k}_{s}"
                A2_ijks[(i, j, k, s)].lowBound = area
                A2_ijks[(i, j, k, s)].upBound = area
                A2_ijks[(i, j, k, s)].cat = "Continuous"

                A1_ijks[(i, j, k, s)].setInitialValue(0)
                A2_ijks[(i, j, k, s)].setInitialValue(area)
            elif i in tent_field:
                A1_ijks[(i, j, k, s)].name = f"A1_{i}_{j}_{k}_{s}"
                A1_ijks[(i, j, k, s)].lowBound = area
                A1_ijks[(i, j, k, s)].upBound = area
                A1_ijks[(i, j, k, s)].cat = "Continuous"

                A2_ijks[(i, j, k, s)].name = f"A2_{i}_{j}_{k}_{s}"
                A2_ijks[(i, j, k, s)].lowBound = 0
                A2_ijks[(i, j, k, s)].upBound = 0
                A2_ijks[(i, j, k, s)].cat = "Continuous"

                A1_ijks[(i, j, k, s)].setInitialValue(area)
                A2_ijks[(i, j, k, s)].setInitialValue(0)


# A_{ijk} = \sum_s A^2_{ijks} + A^1_{ij(k-1)2} + A^1_{ijks}
A_ijk: dict[tuple : pulp.LpVariable] = {}
for i in fields_id:
    for j in crops_id:
        for k in years:
            if k == 2023:
                continue
            sum_A2 = pulp.lpSum([A2_ijks[i, j, k, s] for s in seasons_id])
            A_ijk[(i, j, k)] = sum_A2 + A1_ijks[i, j, k, 1] + A1_ijks[i, j, k - 1, 2]


# 2023 年的 A_{ijk} 已知
for i in fields_id:
    for j in crops_id:
        A_ijk[(i, j, 2023)] = pulp.lpSum([A1_ijks[i, j, 2023, s] for s in seasons_id])



# A_ijks = A^1_{ijks} + A^2_{ijks}
A_ijks: dict[tuple : pulp.LpVariable] = {}
for i in fields_id:
    for j in crops_id:
        for k in years:
            for s in seasons_id:
                A_ijks[(i, j, k, s)] = A1_ijks[(i, j, k, s)] + A2_ijks[(i, j, k, s)]


A_star = {}  # 地块面积

for i in fields_id:
    A_star[i] = stats_2023[stats_2023["Field ID"] == i]["Field Area"].values[0]

# 定义目标函数

In [116]:
# 创建线性规划问题
prob1 = pulp.LpProblem("Crop_Optimization", pulp.LpMaximize)
prob2 = pulp.LpProblem("Crop_Optimization", pulp.LpMaximize)

# Loosely check inequality and not abs function used
def min_loose_list(lst: list):
    min = lst[0]
    for i in range(len(lst)):
        if lst[i] <= min:
            min = lst[i]
    return min


# Loosely check inequality and not abs function used
def max_loose_list(lst: list):
    max = lst[0]
    for i in range(len(lst)):
        if lst[i] >= max:
            max = lst[i]
    return max


# L_1 = \sum_{ijk} \left( \min \left( S_{jk}, Y_{jk} \cdot A_{ijk} \right) \cdot P_{jk} - C_{jk} \cdot A_{ijk} \right)
L1 = pulp.lpSum(
    [
        (
            min_loose_list([S[(j, k)], Y[(j, k)] * A_ijk[(i, j, k)]]) * P[(j, k)]
            - C[(j, k)] * A_ijk[(i, j, k)]
        )
        for i in fields_id
        for j in crops_id
        for k in years
    ]
)

# L_2 = \sum_{ijk} \left( Y_{jk} \cdot A_{ijk} \cdot P_{jk} - C_{jk} \cdot A_{ijk} - 0.5 \cdot \max \left( 0, Y_{jk} \cdot A_{ijk} - S_{jk} \right) \cdot P_{jk} \right)
L2 = pulp.lpSum(
    [
        (
            Y[(j, k)] * A_ijk[(i, j, k)] * P[(j, k)]
            - C[(j, k)] * A_ijk[(i, j, k)]
            - 0.5
            * max_loose_list([0, Y[(j, k)] * A_ijk[(i, j, k)] - S[(j, k)]])
            * P[(j, k)]
        )
        for i in fields_id
        for j in crops_id
        for k in years
    ]
)

# 设置目标函数
prob1 += L1
prob2 += L2


In [117]:
# 约束条件一：一个地块上每种作物合起来的种植面积不超过相应地块的总面积
for i in fields_id:
    for s in seasons_id:
        for k in years:
            prob1 += (
                pulp.lpSum([A_ijks[(i, j, k, s)] for j in T_hat_i_s[(i, s)]])
                <= A_star[i]
            )
            prob2 += (
                pulp.lpSum([A_ijks[(i, j, k, s)] for j in T_hat_i_s[(i, s)]])
                <= A_star[i]
            )

# 约束条件二：每种作物在同一地块（含大棚）都不能连续重茬种植
for i in fields_id:
    for j in crops_id:
        for k in years:
            if k >= years[-1]:  # 最后一年不需要考虑后面的年份
                continue
            if j in T_hat_i_s[(i, 1)] and j not in T_hat_i_s[(i, 1)]:
                prob1 += A_ijks[(i, j, k, 1)] + A_ijks[
                    (i, j, k + 1, 1)
                ] <= min_loose_list([A_ijks[(i, j, k, 1)], A_ijks[(i, j, k + 1, 1)]])
                prob2 += A_ijks[(i, j, k, 1)] + A_ijks[(i, j, k, 1)] <= min_loose_list(
                    [A_ijks[(i, j, k, 1)], A_ijks[(i, j, k + 1, 1)]]
                )
            elif j in T_hat_i_s[(i, 1)] and j in T_hat_i_s[(i, 2)]:
                prob1 += A_ijks[(i, j, k, 1)] + A_ijks[(i, j, k, 2)] <= min_loose_list(
                    [A_ijks[(i, j, k, 1)], A_ijks[(i, j, k, 2)]]
                )
                prob2 += A_ijks[(i, j, k, 1)] + A_ijks[(i, j, k, 2)] <= min_loose_list(
                    [A_ijks[(i, j, k, 1)], A_ijks[(i, j, k, 2)]]
                )
                prob1 += A_ijks[(i, j, k, 2)] + A_ijks[
                    (i, j, k + 1, 1)
                ] <= min_loose_list([A_ijks[(i, j, k, 2)], A_ijks[(i, j, k + 1, 1)]])
                prob2 += A_ijks[(i, j, k, 2)] + A_ijks[
                    (i, j, k + 1, 1)
                ] <= min_loose_list([A_ijks[(i, j, k, 2)], A_ijks[(i, j, k + 1, 1)]])

# 约束条件三：每个地块（含大棚）的所有土地三年内至少种植一次豆类作物
for i in fields_id:
    for j in beans:
        if not j in T_hat_i_s[(i, s)]:
            continue
        for k in years:
            if k >= years[-2]:  # 最后两年不需要考虑后面的年份
                continue
            prob1 += (
                max_loose_list(
                    [
                        A_ijks[(i, j, k, 1)],
                        A_ijks[(i, j, k + 1, 1)],
                        A_ijks[(i, j, k + 2, 1)],
                        A_ijks[(i, j, k, 2)],
                        A_ijks[(i, j, k + 1, 2)],
                        A_ijks[(i, j, k + 2, 2)],
                    ]
                )
                == A_star[i]
            )

            prob2 += (
                max_loose_list(
                    [
                        A_ijks[(i, j, k, 1)],
                        A_ijks[(i, j, k + 1, 1)],
                        A_ijks[(i, j, k + 2, 1)],
                        A_ijks[(i, j, k, 2)],
                        A_ijks[(i, j, k + 1, 2)],
                        A_ijks[(i, j, k + 2, 2)],
                    ]
                )
                == A_star[i]
            )

# M = 0.25
# # 约束条件四：每种作物在单个地块（含大棚）种植的面积不宜太小
# for i in fields_id:
#     for s in seasons_id:
#         for j in crops_id:
#             if not j in T_hat_i_s[(i, s)]:
#                 continue
#             for k in years:
#                 if A_ijks[(i, j, k, s)] != 0:
#                     prob1 += A_ijks[(i, j, k, s)] >= M * A_star[i]
#                     prob2 += A_ijks[(i, j, k, s)] >= M * A_star[i]

# 模型求解

In [118]:
prob1.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/zivmax/CUMCM_2024/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/4010f52c4fc04accb951092852519835-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/4010f52c4fc04accb951092852519835-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10381 COLUMNS
At line 112492 RHS
At line 122869 BOUNDS
At line 158664 ENDATA
Problem MODEL has 10376 rows, 65420 columns and 35888 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
1 infeasibilities
Analysis indicates model infeasible or unbounded
Perturbing problem by 0.001% of 284500 - largest nonzero change 0.28743753 ( 0.016246582%) - largest zero change 0.13921213
0  Obj 3034953 Primal inf 37667.649 (8433) Dual in

-1

In [119]:
prob2.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/zivmax/CUMCM_2024/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/bea6682fa8954ddd88b3cf06bfd6b7e1-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/bea6682fa8954ddd88b3cf06bfd6b7e1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10381 COLUMNS
At line 112492 RHS
At line 122869 BOUNDS
At line 158664 ENDATA
Problem MODEL has 10376 rows, 65420 columns and 35888 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
1 infeasibilities
Analysis indicates model infeasible or unbounded
Perturbing problem by 0.001% of 140750 - largest nonzero change 0.13381482 ( 0.028547399%) - largest zero change 0.068874554
0  Obj 1429674 Primal inf 37667.649 (8433) Dual i

-1

# 整理结果

## 情况一

In [120]:
A1_ijks_1 = {}
A2_ijks_1 = {}
A_ijks_1 = {}

for v in prob1.variables():
    if v.name.startswith("A_star"):
        continue

    if v.name.startswith("A1"):
        name = v.name.replace("A1_", "")
        A1_ijks_1[tuple(map(int, name.split("_")))] = v.varValue

    if v.name.startswith("A2"):
        name = v.name.replace("A2_", "")
        A2_ijks_1[tuple(map(int, name.split("_")))] = v.varValue

# Sort by year, field, crop, season in a single step
A1_ijks_1 = dict(
    sorted(
        A1_ijks_1.items(),
        key=lambda item: (
            int(item[0][2]),  # year
            int(item[0][3]),  # season
            int(item[0][0]),  # field
            int(item[0][1]),  # crop
        ),
    )
)

# for i in A1_ijks_1:
#     print(i, A1_ijks_1[i])

# Sort by year, field, crop, season in a single step
A2_ijks_1 = dict(
    sorted(
        A1_ijks_1.items(),
        key=lambda item: (
            int(item[0][2]),  # year
            int(item[0][0]),  # field
            int(item[0][1]),  # crop
            int(item[0][3]),  # season
        ),
    )
)

for i in A2_ijks_1:
    print(i, A2_ijks_1[i])



(1, 1, 2023, 1) 0.0
(1, 1, 2023, 2) 0.0
(1, 2, 2023, 1) 0.0
(1, 2, 2023, 2) 0.0
(1, 3, 2023, 1) 0.0
(1, 3, 2023, 2) 0.0
(1, 4, 2023, 1) 0.0
(1, 4, 2023, 2) 0.0
(1, 5, 2023, 1) 0.0
(1, 5, 2023, 2) 0.0
(1, 6, 2023, 1) 0.0
(1, 6, 2023, 2) 0.0
(1, 7, 2023, 1) 0.0
(1, 7, 2023, 2) 0.0
(1, 8, 2023, 1) 0.0
(1, 8, 2023, 2) 0.0
(1, 9, 2023, 1) 0.0
(1, 9, 2023, 2) 0.0
(1, 10, 2023, 1) 0.0
(1, 10, 2023, 2) 0.0
(1, 11, 2023, 1) 0.0
(1, 11, 2023, 2) 0.0
(1, 12, 2023, 1) 0.0
(1, 12, 2023, 2) 0.0
(1, 13, 2023, 1) 0.0
(1, 13, 2023, 2) 0.0
(1, 14, 2023, 1) 0.0
(1, 14, 2023, 2) 0.0
(1, 15, 2023, 1) 0.0
(1, 15, 2023, 2) 0.0
(1, 16, 2023, 1) 0.0
(1, 16, 2023, 2) 0.0
(1, 17, 2023, 1) 0.0
(1, 17, 2023, 2) 0.0
(1, 18, 2023, 1) 0.0
(1, 18, 2023, 2) 0.0
(1, 19, 2023, 1) 0.0
(1, 19, 2023, 2) 0.0
(1, 20, 2023, 1) 0.0
(1, 20, 2023, 2) 0.0
(1, 21, 2023, 1) 0.0
(1, 21, 2023, 2) 0.0
(1, 22, 2023, 1) 0.0
(1, 22, 2023, 2) 0.0
(1, 23, 2023, 1) 0.0
(1, 23, 2023, 2) 0.0
(1, 24, 2023, 1) 0.0
(1, 24, 2023, 2) 0.0
(1, 25, 20

In [121]:
import plotly.express as px


for i in A_ijks_1:
    print(i, A_ijks_1[i])

df = pd.DataFrame(A_ijks_1.items(), columns=["Tuple", "Planting Area"])

# Split the tuple into separate columns
df[["Field ID", "Crop ID", "Year", "Season"]] = pd.DataFrame(
    df["Tuple"].tolist(), index=df.index
)

# Drop the original tuple column if not needed
df.drop(columns=["Tuple"], inplace=True)

# Check unique values in the Year column
print(df["Year"].unique())

# # Filter for Year == 2023
# df2023 = df[df["Year"] == 2023]

# # Display the filtered DataFrame
# print(df2023.head())

# fig = px.bar(
#     df[df["Year"] == 2023],
#     x=df[df["Year"] == 2023]["Field ID"] + " " + df[df["Year"] == 2023]["Crop ID"],
#     y="Planting Area",
#     color="Season",
#     barmode="group",
#     title="Planting Area by Field ID, Crop ID, Year and Season",
# )
# fig.show()

ValueError: Columns must be same length as key

## 情况二

In [385]:
A1_ijks_2 = {}
A2_ijks_1 = {}
A_ijks_1 = {}

for v in prob2.variables():
    if v.name.startswith("A_star"):
        continue

    if v.name.startswith("A1"):
        name = v.name.replace("A1_", "")
        A1_ijks_2[tuple(map(int, name.split("_")))] = v.varValue

    if v.name.startswith("A2"):
        name = v.name.replace("A2_", "")
        A2_ijks_1[tuple(map(int, name.split("_")))] = v.varValue

# Sort by year, field, crop, season in a single step
A1_ijks_2 = dict(
    sorted(
        A1_ijks_2.items(),
        key=lambda item: (
            int(item[0][2]),  # year
            int(item[0][0]),  # field
            int(item[0][1]),  # crop
            int(item[0][3]),  # season
        ),
    )
)

# Sort by year, field, crop, season in a single step
A2_ijks_1 = dict(
    sorted(
        A1_ijks_2.items(),
        key=lambda item: (
            int(item[0][2]),  # year
            int(item[0][0]),  # field
            int(item[0][1]),  # crop
            int(item[0][3]),  # season
        ),
    )
)

for k in years:
    for i in fields_id:
        for j in crops_id:
            for s in seasons_id:
                A_ijks_1[(i, j, k, s)] = (
                    A1_ijks_2[(i, j, k, s)] + A2_ijks_1[(i, j, k, s)]
                )